In [1]:
# !pip install datasets

In [2]:


from datasets import load_dataset
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import os
import json
import torch.nn.functional as F
from pathlib import Path
from shutil import copyfile
from typing import List, Dict, Union, Callable, Optional
from copy import deepcopy

import numpy as np
from scipy.spatial.transform import Rotation as R

import matplotlib as plt

import numpy as np

import os

import pdb
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tqdm import tqdm

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
x_size = 120
y_size = 90
grid_size = 1

In [4]:
train_path = fr"C:\Users\yuval\training_batch_{grid_size}_meter_grid.pt"
labels_path = fr"C:\Users\yuval\label_batch_{grid_size}_meter_grid.pt"

train = torch.load(train_path)
labels = torch.load(labels_path)

In [5]:
def process_train_tensor(tensor):
    """
    Processes a tensor based on the provided criteria.

    Args:
      tensor: The input tensor.

    Returns:
      A list of tensors.
    """
    return (tensor[0, :, :]) / 1000
processed_train = torch.empty(0,x_size, y_size)
for i in range(train.size(0)):
  new_train = process_train_tensor(train[i])
  processed_train = torch.cat((processed_train, new_train.unsqueeze(0)), dim=0)
processed_train.size()

torch.Size([820, 120, 90])

In [6]:
processed_labels = torch.empty(0,x_size, y_size)
def process_labels(labels):
  sample = labels.numpy()
  # Assuming sample has shape (3, 40, 30)
  max_values = np.max(sample, axis=0)
  processed_labels = (max_values > 0).astype(int) # Check if any value in the 3 tensors is greater than 0
  processed_labels = torch.tensor(processed_labels)
  return processed_labels

for i in range(labels.size(0)):
  new = process_labels(labels[i])
  processed_labels = torch.cat((processed_labels, new.unsqueeze(0)), dim=0)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

learning_rate = 0.0001
num_epochs = 3
batch_size = 16

# Define the neural network architecture
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        # Changed floating-point division to integer division
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        # Changed floating-point division to integer division
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
input_size = x_size * y_size
hidden_size = x_size*y_size*32  # Adjust as needed
output_size = x_size * y_size   # Binary classification (0 or 1)

# Create the model
model = SimpleClassifier(input_size, hidden_size, output_size).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(1000, dtype=torch.float32))  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a TensorDataset
dataset = TensorDataset(processed_train.float(), processed_labels.float())

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt

# Training loop with validation
train_loss_history = []
val_loss_history = []
for epoch in tqdm(range(num_epochs)):
    train_epoch_loss = 0
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        data = data.to(device)
        # Get the actual batch size of the current batch
        current_batch_size = data.size(0)
        # Use the actual batch size for reshaping
        target = target.view(current_batch_size, x_size, y_size).to(device)
        outputs = model(data).view(current_batch_size, x_size, y_size).to(device)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        train_epoch_loss += loss.item()

    train_loss_history.append(train_epoch_loss / len(train_loader))

    val_epoch_loss = 0
    model.eval()  # Set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        for data, target in val_loader:
            # Get the actual batch size of the current batch
            current_batch_size = data.size(0)  # Add this line
            # Use the actual batch size for reshaping
            target = target.view(current_batch_size, x_size, y_size).to(device)  # Use current_batch_size
            data = data.to(device)
            outputs = model(data).view(current_batch_size, x_size, y_size).to(device)  # Use current_batch_size
            loss = criterion(outputs, target)
            val_epoch_loss += loss.item()
    val_loss_history.append(val_epoch_loss / len(val_loader))

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_epoch_loss / len(train_loader):.4f}, Val Loss: {val_epoch_loss / len(val_loader):.4f}')

# Plot the loss
plt.plot(train_loss_history, label='Train Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# prompt: save SimpleClassifier state dict so I will be able to use it

# Save the model's state dictionary
torch.save(model.state_dict(), fr'C:\Users\yuval\simple_classifier_state_dict_(grid_size)_meters_grid.pth')


In [ ]:
# prompt: calculate accuracy, recall, precision

# Evaluation on the validation set
model.eval()  # Set the model to evaluation mode
all_predictions = []
all_targets = []

with torch.no_grad():
    for data, target in val_loader:
        data = data.to(device)
        # Get the actual batch size of the current batch
        current_batch_size = data.size(0)
        target = target.view(current_batch_size, x_size, y_size).to(device)

        outputs = model(data).view(current_batch_size, x_size, y_size).to(device)

        # Apply threshold to get binary predictions
        predicted = (outputs > 0.07).int()

        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_targets.extend(target.cpu().numpy().flatten())

# Calculate metrics
accuracy = accuracy_score(all_targets, all_predictions)
precision = precision_score(all_targets, all_predictions)
recall = recall_score(all_targets, all_predictions)
auc = roc_auc_score(all_targets, all_predictions)
conf_matrix = confusion_matrix(all_targets, all_predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print (f"AUC: {auc:.4f}")
print("Confusion Matrix:")
conf_matrix